In [ ]:

#  Setting display options 

import pandas as pd
   #
pd.set_option("display.width", 480)

#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML(""))

from tabulate import tabulate

print("--")


#  Setup stuff: Connectivity

In [ ]:

from katana import remote
from katana.remote import import_data

my_client = remote.Client()

print(my_client)


In [ ]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


In [ ]:

#  CONNECT TO GRAPH

my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)


#  Analytics: Why we are here

In [ ]:

#  First neighborhood
#
#     (No rows found)

l_result = my_graph.query("""

   MATCH (n: Person) - [r] -> (m) 
   WHERE
      m.city = 'Houston'
   RETURN *

   """, contextualize=True)

l_result.view()


In [ ]:

#  All neighborhood depths
#
#     (Currently: error)

l_result = my_graph.query("""

   MATCH (n: Person) - [r *] -> (m) 
   WHERE
      m.city = 'Houston'
   RETURN n, r, m

   """, contextualize=True)

l_result.view()


# display(print(l_result))
# 
#  Outputted value,
#
#  OperationError: FZSoWndtUBvHNquRkos1tkw5DS9CwvH6NF4X5Ady2WfZ-43CxYsTPwbyPAsoQQ backtrace (QueryClient.cpp:575):
#  opgraph check failed (OpGraph.cpp:564): Unimplemented Features Pass (OpGraphErrorChecker.cpp:12):
#  backtrace (UnimplementedFeaturesChecker.cpp:376): backtrace (OpGraph.cpp:822):
#  backtrace (OpGraph.cpp:822): backtrace (OpGraph.cpp:798):
#  returning named variable paths not yet supported (UnimplementedFeaturesChecker.cpp:197): InternalError:NotImplemented: InternalError:NotImplemented



In [7]:

#  How Persons relate to Houston ..
#
#     P_101   Justin
#     P_102   Anand
#     P_103   Brian
#     P_104   Sameer

#  The 

l_result = my_graph.query("""

   MATCH (n: Person) - [*] - > (m) 
   WHERE
      m.city = 'Houston'
   RETURN *

   """, contextualize=True)

l_result.view()


# display(print(l_result))
# 
#      n.internal_id  n.labels n.LABEL   n.id  n.name n.type  m.internal_id       m.labels      m.LABEL   m.city   m.id m.name m.type
#      
#  0               3  [Person]  Person  P_104  Sameer   node             23     [Reviewer]     Reviewer  Houston  R_102  Allen   node
#  2               3  [Person]  Person  P_104  Sameer   node             21  [Dist_Center]  Dist_Center  Houston  D_102    NaN   node
#  4               3  [Person]  Person  P_104  Sameer   node             21  [Dist_Center]  Dist_Center  Houston  D_102    NaN   node
#  10              3  [Person]  Person  P_104  Sameer   node             19        [Store]        Store  Houston  S_104    NaN   node
#  12              3  [Person]  Person  P_104  Sameer   node             19        [Store]        Store  Houston  S_104    NaN   node
#  
#  1               1  [Person]  Person  P_102   Anand   node             21  [Dist_Center]  Dist_Center  Houston  D_102    NaN   node
#  3               1  [Person]  Person  P_102   Anand   node             21  [Dist_Center]  Dist_Center  Houston  D_102    NaN   node
#  9               1  [Person]  Person  P_102   Anand   node             19        [Store]        Store  Houston  S_104    NaN   node
#  11              1  [Person]  Person  P_102   Anand   node             19        [Store]        Store  Houston  S_104    NaN   node
#  
#  5               0  [Person]  Person  P_101  Justin   node             19        [Store]        Store  Houston  S_104    NaN   node
#  7               0  [Person]  Person  P_101  Justin   node             19        [Store]        Store  Houston  S_104    NaN   node
#  
#  6               2  [Person]  Person  P_103   Brian   node             19        [Store]        Store  Houston  S_104    NaN   node
#  8               2  [Person]  Person  P_103   Brian   node             19        [Store]        Store  Houston  S_104    NaN   node



          0/? [?op/s]

          0/? [?op/s]

In [ ]:
# ######################################################

In [ ]:

#  Can we filter out where date_shipped has already taken place ?
#

l_result = my_graph.query("""

   MATCH (n: Person) - [*] - > (m) 
   WHERE
      m.city = 'Houston'
   RETURN m.date_shipped

   """, contextualize=True)

l_result.view()


#  Outputted values,
#
#  m.date_shipped
#  None
#  None
#  None
#  None
#  None
#  None
#  None
#  None
#  None


In [ ]:

#  This fails gracefully after 60 (?) seconds ..

l_result = my_graph.query("""

   MATCH (n: Person) - [*] - (o) - [*] - > (m) 
   WHERE
      m.city = 'Houston'
   RETURN o.date_shipped

   """, contextualize=True)

l_result.view()


#  Outputted values,
#
#  OperationError: 5BsJxxpX5GRqbGGukKZiyEnNn6RBwjzaCVboNZQfs2o5-5B5NmeiqeqDiGVtQb 


In [ ]:

l_result = my_graph.query("""

   MATCH (n: Person) - [p: PLACED] - (o: Order) - [*] - > (m) 
   WHERE
      m.city = 'Houston'
   AND
      NOT EXISTS (o.date_shipped)
   RETURN *

   """, contextualize=True)

l_result.view()


#  Doesn't achieve desired results; data was entered as an empty string


In [ ]:

#  Filter on date_shipped ..
#

l_result = my_graph.query("""

   MATCH (n: Person) - [p: PLACED] - (o: Order) - [*] - > (m) 
   WHERE
      m.city = 'Houston'
   AND
      SIZE (o.date_shipped) < 1 
   RETURN *

   """, contextualize=True)

l_result.view()
